Life Settlement Underwriting Model - This notebook generates a risk score based on input parameters received in JS format.

Import and define functions

In [1]:
import pandas as PD
import numpy as NP
import json as JS
import time as TM
from datetime import datetime as DT
from typing import Dict, Any, Tuple

START_TIME = TM.perf_counter()

def YEARS_BETWEEN(early_date, later_date):
    return PD.Timestamp(later_date).year - PD.Timestamp(early_date).year - ((PD.Timestamp(later_date).month, PD.Timestamp(later_date).day) < (PD.Timestamp(early_date).month, PD.Timestamp(early_date).day))

def AGE_AT(date_of_birth, reference_date):
    return YEARS_BETWEEN(date_of_birth, reference_date)

Define Input Schema and Validation

In [2]:
# Define expected JS schema for inputs

INPUT_SCHEMA = {
    "demographics": {
        "date_of_birth": "YYYY-MM-DD",
        "gender": "M|F|Other",
        "issue_date": "YYYY-MM-DD",
        "current_date": "YYYY-MM-DD"
    },
    "policy_info": {
        "face_amount": "numeric",
        "issue_state": "two-letter state code",
        "product_type": "Term|WL|UL|IUL",
        "years_to_maturity": "numeric"
    },
    "health_factors": {
        "medical_conditions": ["list of conditions"],
        "smoking_status": "Yes|No",
        "height_inches": "numeric",
        "weight_pounds": "numeric",
        "health_rating": "Excellent|Good|Fair|Poor"
    },
    "financial_factors": {
        "annual_income": "numeric",
        "debt_level": "numeric",
        "liquid_assets": "numeric"
    }
}

def VALIDATE_INPUT(input_data: Dict[str, Any]) -> Tuple[bool, str]:
    """Validate input JS structure and required fields"""
    required_demographics = ["date_of_birth", "gender", "issue_date", "current_date"]
    required_policy = ["face_amount", "issue_state", "product_type", "years_to_maturity"]
    required_health = ["medical_conditions", "smoking_status", "height_inches", "weight_pounds", "health_rating"]
    required_financial = ["annual_income", "debt_level", "liquid_assets"]
    
    # Check demographics
    if "demographics" not in input_data:
        return False, "Missing 'demographics' section"
    for field in required_demographics:
        if field not in input_data["demographics"]:
            return False, f"Missing required field: demographics.{field}"
    
    # Check policy_info
    if "policy_info" not in input_data:
        return False, "Missing 'policy_info' section"
    for field in required_policy:
        if field not in input_data["policy_info"]:
            return False, f"Missing required field: policy_info.{field}"
    
    # Check health_factors
    if "health_factors" not in input_data:
        return False, "Missing 'health_factors' section"
    for field in required_health:
        if field not in input_data["health_factors"]:
            return False, f"Missing required field: health_factors.{field}"
    
    # Check financial_factors
    if "financial_factors" not in input_data:
        return False, "Missing 'financial_factors' section"
    for field in required_financial:
        if field not in input_data["financial_factors"]:
            return False, f"Missing required field: financial_factors.{field}"
    
    return True, "Validation passed"

print("Schema defined and validation function created")

Schema defined and validation function created


Load and Parse JS Input

In [3]:
def LOAD_JS_FILE(file_path: str) -> Dict[str, Any]:
    """Load JS input from file"""
    try:
        with open(file_path, 'r') as f:
            return JS.load(f)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found")
        return {}
    except JS.JSONDecodeError:
        print(f"Error: File '{file_path}' is not valid JS")
        return {}

def PARSE_JS_INPUT(json_string: str) -> Dict[str, Any]:
    """Parse JS input from string"""
    try:
        return JS.loads(json_string)
    except JS.JSONDecodeError as e:
        print(f"Error parsing JS: {e}")
        return {}

# Example JS input data
EXAMPLE_JS_INPUT = {
    "demographics": {
        "date_of_birth": "1962-03-15",
        "gender": "M",
        "issue_date": "2010-06-01",
        "current_date": "2026-02-06"
    },
    "policy_info": {
        "face_amount": 1000000,
        "issue_state": "CA",
        "product_type": "Term",
        "years_to_maturity": 15
    },
    "health_factors": {
        "medical_conditions": ["Hypertension", "Type 2 Diabetes"],
        "smoking_status": "No",
        "height_inches": 70,
        "weight_pounds": 185,
        "health_rating": "Fair"
    },
    "financial_factors": {
        "annual_income": 85000,
        "debt_level": 150000,
        "liquid_assets": 250000
    }
}

# Validate the example input
is_valid, validation_msg = VALIDATE_INPUT(EXAMPLE_JS_INPUT)
print(f"Input Validation: {validation_msg}")

# Extract input data
DEMOGRAPHICS = EXAMPLE_JS_INPUT["demographics"]
POLICY_INFO = EXAMPLE_JS_INPUT["policy_info"]
HEALTH_FACTORS = EXAMPLE_JS_INPUT["health_factors"]
FINANCIAL_FACTORS = EXAMPLE_JS_INPUT["financial_factors"]

Input Validation: Validation passed


Calculate Health Risk Factors

In [4]:
def CALCULATE_BMI(height_inches: float, weight_pounds: float) -> float:
    """Calculate Body Mass Index from height and weight"""
    height_meters = height_inches * 0.0254
    weight_kg = weight_pounds * 0.453592
    return weight_kg / (height_meters ** 2)

def CALCULATE_BMI_RISK(bmi: float) -> float:
    """Generate BMI risk score (0-100 scale, 100 = highest risk)"""
    if bmi < 18.5:
        return 20.0  # Underweight
    elif bmi < 25:
        return 10.0  # Normal
    elif bmi < 30:
        return 20.0  # Overweight
    elif bmi < 35:
        return 35.0  # Obese Class I
    else:
        return 50.0  # Obese Class II+

def CALCULATE_AGE_RISK(current_age: int) -> float:
    """Generate age-based risk score (0-100 scale)"""
    if current_age < 50:
        return 5.0
    elif current_age < 60:
        return 15.0
    elif current_age < 70:
        return 30.0
    elif current_age < 80:
        return 50.0
    else:
        return 70.0

def CALCULATE_MEDICAL_CONDITION_RISK(conditions: list) -> float:
    """Calculate risk based on medical conditions"""
    condition_risk_map = {
        "Hypertension": 5.0,
        "Type 2 Diabetes": 10.0,
        "Type 1 Diabetes": 15.0,
        "Heart Disease": 25.0,
        "Cancer": 30.0,
        "Stroke History": 20.0,
        "COPD": 20.0,
        "Kidney Disease": 15.0,
        "Liver Disease": 20.0,
        "AIDS": 35.0
    }
    
    total_risk = 0.0
    for condition in conditions:
        total_risk += condition_risk_map.get(condition, 3.0)
    
    # Cap at 80 to leave room for other factors
    return min(total_risk, 80.0)

def CALCULATE_HEALTH_RATING_RISK(health_rating: str) -> float:
    """Convert health rating to risk score"""
    rating_map = {
        "Excellent": 5.0,
        "Good": 15.0,
        "Fair": 35.0,
        "Poor": 60.0
    }
    return rating_map.get(health_rating, 25.0)

def CALCULATE_SMOKING_RISK(smoking_status: str) -> float:
    """Calculate smoking-related risk"""
    return 15.0 if smoking_status.lower() == "yes" else 0.0

# Calculate health risk factors
CURRENT_AGE = AGE_AT(DEMOGRAPHICS["date_of_birth"], DEMOGRAPHICS["current_date"])
BMI = CALCULATE_BMI(HEALTH_FACTORS["height_inches"], HEALTH_FACTORS["weight_pounds"])

HEALTH_RISK_AGE = CALCULATE_AGE_RISK(CURRENT_AGE)
HEALTH_RISK_BMI = CALCULATE_BMI_RISK(BMI)
HEALTH_RISK_CONDITIONS = CALCULATE_MEDICAL_CONDITION_RISK(HEALTH_FACTORS["medical_conditions"])
HEALTH_RISK_RATING = CALCULATE_HEALTH_RATING_RISK(HEALTH_FACTORS["health_rating"])
HEALTH_RISK_SMOKING = CALCULATE_SMOKING_RISK(HEALTH_FACTORS["smoking_status"])

print(f"Current Age: {CURRENT_AGE} years")
print(f"BMI: {BMI:.2f}")
print(f"Age Risk: {HEALTH_RISK_AGE:.1f}")
print(f"BMI Risk: {HEALTH_RISK_BMI:.1f}")
print(f"Medical Conditions Risk: {HEALTH_RISK_CONDITIONS:.1f}")
print(f"Health Rating Risk: {HEALTH_RISK_RATING:.1f}")
print(f"Smoking Risk: {HEALTH_RISK_SMOKING:.1f}")

Current Age: 63 years
BMI: 26.54
Age Risk: 30.0
BMI Risk: 20.0
Medical Conditions Risk: 15.0
Health Rating Risk: 35.0
Smoking Risk: 0.0


Calculate Financial Risk Factors

In [5]:
def CALCULATE_DEBT_TO_INCOME_RATIO(annual_income: float, debt_level: float) -> float:
    """Calculate debt to income ratio"""
    if annual_income <= 0:
        return 100.0
    return (debt_level / annual_income) * 100

def CALCULATE_DEBT_RISK(debt_to_income_ratio: float) -> float:
    """Generate debt risk score based on debt-to-income ratio"""
    if debt_to_income_ratio < 50:
        return 10.0
    elif debt_to_income_ratio < 100:
        return 25.0
    elif debt_to_income_ratio < 150:
        return 40.0
    elif debt_to_income_ratio < 200:
        return 55.0
    else:
        return 70.0

def CALCULATE_LIQUIDITY_RATIO(liquid_assets: float, debt_level: float) -> float:
    """Calculate liquidity ratio (assets to debt)"""
    if debt_level <= 0:
        return 100.0
    return (liquid_assets / debt_level)

def CALCULATE_LIQUIDITY_RISK(liquidity_ratio: float) -> float:
    """Generate liquidity risk score - higher ratio is better (lower risk)"""
    if liquidity_ratio >= 2.0:
        return 5.0
    elif liquidity_ratio >= 1.0:
        return 15.0
    elif liquidity_ratio >= 0.5:
        return 35.0
    else:
        return 60.0

def CALCULATE_FACE_AMOUNT_RISK(face_amount: float) -> float:
    """Generate risk based on face amount (very large amounts may indicate adverse selection)"""
    if face_amount < 250000:
        return 10.0
    elif face_amount < 1000000:
        return 15.0
    elif face_amount < 5000000:
        return 20.0
    else:
        return 30.0

# Calculate financial risk factors
DEBT_TO_INCOME_RATIO = CALCULATE_DEBT_TO_INCOME_RATIO(FINANCIAL_FACTORS["annual_income"], FINANCIAL_FACTORS["debt_level"])
LIQUIDITY_RATIO = CALCULATE_LIQUIDITY_RATIO(FINANCIAL_FACTORS["liquid_assets"], FINANCIAL_FACTORS["debt_level"])

FINANCIAL_RISK_DEBT = CALCULATE_DEBT_RISK(DEBT_TO_INCOME_RATIO)
FINANCIAL_RISK_LIQUIDITY = CALCULATE_LIQUIDITY_RISK(LIQUIDITY_RATIO)
FINANCIAL_RISK_FACE_AMOUNT = CALCULATE_FACE_AMOUNT_RISK(POLICY_INFO["face_amount"])

print(f"Debt-to-Income Ratio: {DEBT_TO_INCOME_RATIO:.1f}%")
print(f"Liquidity Ratio: {LIQUIDITY_RATIO:.2f}")
print(f"Debt Risk: {FINANCIAL_RISK_DEBT:.1f}")
print(f"Liquidity Risk: {FINANCIAL_RISK_LIQUIDITY:.1f}")
print(f"Face Amount Risk: {FINANCIAL_RISK_FACE_AMOUNT:.1f}")

Debt-to-Income Ratio: 176.5%
Liquidity Ratio: 1.67
Debt Risk: 55.0
Liquidity Risk: 15.0
Face Amount Risk: 20.0


Calculate Mortality Probability

In [6]:
def CALCULATE_BASE_MORTALITY(age: int, gender: str) -> float:
    """Calculate base mortality rate using simple actuarial model"""
    # Simplified GAM83 model approximation
    if gender.lower() == "m":
        q_rates = {
            50: 0.0053, 55: 0.0079, 60: 0.0115, 65: 0.0168, 70: 0.0250,
            75: 0.0369, 80: 0.0540, 85: 0.0780, 90: 0.1100, 95: 0.1550
        }
    else:
        q_rates = {
            50: 0.0032, 55: 0.0048, 60: 0.0070, 65: 0.0105, 70: 0.0155,
            75: 0.0230, 80: 0.0350, 85: 0.0520, 90: 0.0780, 95: 0.1150
        }
    
    # Linear interpolation for ages between key points
    ages = sorted(q_rates.keys())
    if age <= ages[0]:
        return q_rates[ages[0]]
    if age >= ages[-1]:
        return q_rates[ages[-1]]
    
    for i in range(len(ages) - 1):
        if ages[i] <= age < ages[i + 1]:
            age_diff = ages[i + 1] - ages[i]
            rate_diff = q_rates[ages[i + 1]] - q_rates[ages[i]]
            return q_rates[ages[i]] + (age - ages[i]) * (rate_diff / age_diff)
    
    return q_rates[ages[-1]]

def CALCULATE_ADJUSTED_MORTALITY(base_mortality: float, health_risk: float) -> float:
    """Adjust mortality rate based on health risk factors"""
    # Health risk adjustment: 0-20% increase based on health risk score (0-100)
    health_adjustment = 1 + (health_risk / 100) * 0.20
    return base_mortality * health_adjustment

def CALCULATE_SURVIVAL_PROBABILITY(adjusted_mortality: float, years: int) -> float:
    """Calculate probability of surviving N years"""
    px = 1 - adjusted_mortality
    return px ** years

# Calculate mortality probability
ISSUE_AGE = AGE_AT(DEMOGRAPHICS["date_of_birth"], DEMOGRAPHICS["issue_date"])
YEARS_SINCE_ISSUE = YEARS_BETWEEN(DEMOGRAPHICS["issue_date"], DEMOGRAPHICS["current_date"])
YEARS_TO_MATURITY = POLICY_INFO["years_to_maturity"]

BASE_MORTALITY_RATE = CALCULATE_BASE_MORTALITY(CURRENT_AGE, DEMOGRAPHICS["gender"])

# Combined health risk
TOTAL_HEALTH_RISK = min(80.0, (HEALTH_RISK_AGE + HEALTH_RISK_BMI + HEALTH_RISK_CONDITIONS + 
                                 HEALTH_RISK_RATING + HEALTH_RISK_SMOKING) / 5)

ADJUSTED_MORTALITY_RATE = CALCULATE_ADJUSTED_MORTALITY(BASE_MORTALITY_RATE, TOTAL_HEALTH_RISK)
SURVIVAL_PROB_10YEARS = CALCULATE_SURVIVAL_PROBABILITY(ADJUSTED_MORTALITY_RATE, 10)
SURVIVAL_PROB_MATURITY = CALCULATE_SURVIVAL_PROBABILITY(ADJUSTED_MORTALITY_RATE, YEARS_TO_MATURITY)

print(f"Issue Age: {ISSUE_AGE}")
print(f"Years Since Issue: {YEARS_SINCE_ISSUE}")
print(f"Base Mortality Rate: {BASE_MORTALITY_RATE:.4f}")
print(f"Total Health Risk Score: {TOTAL_HEALTH_RISK:.1f}")
print(f"Adjusted Mortality Rate: {ADJUSTED_MORTALITY_RATE:.4f}")
print(f"10-Year Survival Probability: {SURVIVAL_PROB_10YEARS:.2%}")
print(f"Survival to Maturity ({YEARS_TO_MATURITY} years): {SURVIVAL_PROB_MATURITY:.2%}")

Issue Age: 48
Years Since Issue: 15
Base Mortality Rate: 0.0147
Total Health Risk Score: 20.0
Adjusted Mortality Rate: 0.0153
10-Year Survival Probability: 85.74%
Survival to Maturity (15 years): 79.39%


Generate Risk Score

In [7]:
def CALCULATE_MORTALITY_RISK_SCORE(adjusted_mortality_rate: float, survival_probability: float) -> float:
    """Convert mortality and survival metrics to risk score (0-100)"""
    # Higher mortality = higher risk
    mortality_risk = min(100.0, adjusted_mortality_rate * 10000)
    # Lower survival = higher risk
    survival_risk = (1 - survival_probability) * 100
    return (mortality_risk + survival_risk) / 2

def CALCULATE_POLICY_RISK_SCORE(years_to_maturity: int, years_since_issue: int) -> float:
    """Calculate policy structure risk based on time factors"""
    remaining_years = years_to_maturity
    
    if remaining_years < 2:
        return 70.0  # Short duration = high risk (more mortality exposure)
    elif remaining_years < 5:
        return 50.0
    elif remaining_years < 10:
        return 30.0
    elif remaining_years < 20:
        return 20.0
    else:
        return 10.0

def CALCULATE_COMPREHENSIVE_RISK_SCORE(health_risk_avg: float, financial_risk_avg: float, 
                                       mortality_risk: float, policy_risk: float) -> float:
    """Calculate comprehensive underwriting risk score (0-100)"""
    # Weighted combination of all risk factors
    # Health: 35%, Financial: 25%, Mortality: 30%, Policy: 10%
    
    comprehensive_score = (
        health_risk_avg * 0.35 +
        financial_risk_avg * 0.25 +
        mortality_risk * 0.30 +
        policy_risk * 0.10
    )
    
    return min(100.0, comprehensive_score)

def GENERATE_RISK_CLASSIFICATION(risk_score: float) -> str:
    """Convert numerical risk score to classification"""
    if risk_score < 20:
        return "VERY_LOW"
    elif risk_score < 35:
        return "LOW"
    elif risk_score < 50:
        return "MODERATE"
    elif risk_score < 70:
        return "HIGH"
    else:
        return "VERY_HIGH"

# Calculate component risk averages
HEALTH_RISK_AVERAGE = (HEALTH_RISK_AGE + HEALTH_RISK_BMI + HEALTH_RISK_CONDITIONS + 
                       HEALTH_RISK_RATING + HEALTH_RISK_SMOKING) / 5

FINANCIAL_RISK_AVERAGE = (FINANCIAL_RISK_DEBT + FINANCIAL_RISK_LIQUIDITY + FINANCIAL_RISK_FACE_AMOUNT) / 3

MORTALITY_RISK_SCORE = CALCULATE_MORTALITY_RISK_SCORE(ADJUSTED_MORTALITY_RATE, SURVIVAL_PROB_10YEARS)

POLICY_RISK_SCORE = CALCULATE_POLICY_RISK_SCORE(YEARS_TO_MATURITY, YEARS_SINCE_ISSUE)

# Calculate final comprehensive risk score
FINAL_RISK_SCORE = CALCULATE_COMPREHENSIVE_RISK_SCORE(HEALTH_RISK_AVERAGE, FINANCIAL_RISK_AVERAGE, 
                                                       MORTALITY_RISK_SCORE, POLICY_RISK_SCORE)

RISK_CLASSIFICATION = GENERATE_RISK_CLASSIFICATION(FINAL_RISK_SCORE)

print(f"\n{'='*60}")
print(f"RISK SCORE COMPONENTS")
print(f"{'='*60}")
print(f"Health Risk Average: {HEALTH_RISK_AVERAGE:.1f}")
print(f"Financial Risk Average: {FINANCIAL_RISK_AVERAGE:.1f}")
print(f"Mortality Risk Score: {MORTALITY_RISK_SCORE:.1f}")
print(f"Policy Risk Score: {POLICY_RISK_SCORE:.1f}")


RISK SCORE COMPONENTS
Health Risk Average: 20.0
Financial Risk Average: 30.0
Mortality Risk Score: 57.1
Policy Risk Score: 20.0


Format and Return Results

In [8]:
def FORMAT_UNDERWRITING_RESULTS(demographics: Dict, policy: Dict, health_metrics: Dict, 
                                 financial_metrics: Dict, risk_components: Dict,
                                 final_score: float, classification: str) -> Dict[str, Any]:
    """Format all underwriting results into a structured output"""
    
    results = {
        "underwriting_summary": {
            "final_risk_score": round(final_score, 2),
            "risk_classification": classification,
            "evaluation_date": DT.now().strftime("%Y-%m-%d"),
            "approved": "Yes" if final_score < 50 else "No" if final_score >= 70 else "Conditional"
        },
        "applicant_profile": {
            "age": demographics["current_age"],
            "gender": demographics["gender"],
            "issue_age": demographics["issue_age"],
            "years_since_issue": demographics["years_since_issue"]
        },
        "policy_details": {
            "face_amount": policy["face_amount"],
            "product_type": policy["product_type"],
            "years_to_maturity": policy["years_to_maturity"],
            "issue_state": policy["issue_state"]
        },
        "health_assessment": health_metrics,
        "financial_assessment": financial_metrics,
        "risk_component_scores": risk_components,
        "mortality_analysis": {
            "base_mortality_rate": round(demographics["base_mortality_rate"], 6),
            "adjusted_mortality_rate": round(demographics["adjusted_mortality_rate"], 6),
            "10_year_survival_probability": round(demographics["survival_prob_10y"], 4)
        }
    }
    
    return results

# Build results dictionary with all health metrics
HEALTH_ASSESSMENT = {
    "bmi": round(BMI, 2),
    "bmi_risk_score": round(HEALTH_RISK_BMI, 1),
    "medical_conditions": HEALTH_FACTORS["medical_conditions"],
    "medical_conditions_risk": round(HEALTH_RISK_CONDITIONS, 1),
    "health_rating": HEALTH_FACTORS["health_rating"],
    "health_rating_risk": round(HEALTH_RISK_RATING, 1),
    "smoking_status": HEALTH_FACTORS["smoking_status"],
    "smoking_risk": round(HEALTH_RISK_SMOKING, 1),
    "age_risk_score": round(HEALTH_RISK_AGE, 1),
    "overall_health_risk": round(HEALTH_RISK_AVERAGE, 1)
}

# Build financial metrics dictionary
FINANCIAL_ASSESSMENT = {
    "annual_income": FINANCIAL_FACTORS["annual_income"],
    "debt_level": FINANCIAL_FACTORS["debt_level"],
    "liquid_assets": FINANCIAL_FACTORS["liquid_assets"],
    "debt_to_income_ratio": round(DEBT_TO_INCOME_RATIO, 2),
    "debt_to_income_risk": round(FINANCIAL_RISK_DEBT, 1),
    "liquidity_ratio": round(LIQUIDITY_RATIO, 2),
    "liquidity_risk": round(FINANCIAL_RISK_LIQUIDITY, 1),
    "face_amount_risk": round(FINANCIAL_RISK_FACE_AMOUNT, 1),
    "overall_financial_risk": round(FINANCIAL_RISK_AVERAGE, 1)
}

# Build risk components
RISK_COMPONENTS = {
    "health_risk": round(HEALTH_RISK_AVERAGE, 1),
    "financial_risk": round(FINANCIAL_RISK_AVERAGE, 1),
    "mortality_risk": round(MORTALITY_RISK_SCORE, 1),
    "policy_risk": round(POLICY_RISK_SCORE, 1)
}

# Build demographics for results
DEMOGRAPHICS_FOR_RESULTS = {
    "current_age": CURRENT_AGE,
    "gender": DEMOGRAPHICS["gender"],
    "issue_age": ISSUE_AGE,
    "years_since_issue": YEARS_SINCE_ISSUE,
    "base_mortality_rate": BASE_MORTALITY_RATE,
    "adjusted_mortality_rate": ADJUSTED_MORTALITY_RATE,
    "survival_prob_10y": SURVIVAL_PROB_10YEARS
}

# Format final results
UNDERWRITING_RESULTS = FORMAT_UNDERWRITING_RESULTS(
    DEMOGRAPHICS_FOR_RESULTS,
    POLICY_INFO,
    HEALTH_ASSESSMENT,
    FINANCIAL_ASSESSMENT,
    RISK_COMPONENTS,
    FINAL_RISK_SCORE,
    RISK_CLASSIFICATION
)

# Display results
print(f"\n{'='*80}")
print(f"FINAL UNDERWRITING RISK SCORE AND ASSESSMENT")
print(f"{'='*80}")
print(f"Risk Score: {FINAL_RISK_SCORE:.2f}/100")
print(f"Classification: {RISK_CLASSIFICATION}")
print(f"Approval Status: {UNDERWRITING_RESULTS['underwriting_summary']['approved']}")
print(f"\nDetailed Results (JS format):")
print(JS.dumps(UNDERWRITING_RESULTS, indent=2))


FINAL UNDERWRITING RISK SCORE AND ASSESSMENT
Risk Score: 33.64/100
Classification: LOW
Approval Status: Yes

Detailed Results (JS format):
{
  "underwriting_summary": {
    "final_risk_score": 33.64,
    "risk_classification": "LOW",
    "evaluation_date": "2026-02-06",
    "approved": "Yes"
  },
  "applicant_profile": {
    "age": 63,
    "gender": "M",
    "issue_age": 48,
    "years_since_issue": 15
  },
  "policy_details": {
    "face_amount": 1000000,
    "product_type": "Term",
    "years_to_maturity": 15,
    "issue_state": "CA"
  },
  "health_assessment": {
    "bmi": 26.54,
    "bmi_risk_score": 20.0,
    "medical_conditions": [
      "Hypertension",
      "Type 2 Diabetes"
    ],
    "medical_conditions_risk": 15.0,
    "health_rating": "Fair",
    "health_rating_risk": 35.0,
    "smoking_status": "No",
    "smoking_risk": 0.0,
    "age_risk_score": 30.0,
    "overall_health_risk": 20.0
  },
  "financial_assessment": {
    "annual_income": 85000,
    "debt_level": 150000,
  

Total Execution Time

In [9]:
END_TIME = TM.perf_counter()
print(f"\nExecution Time: {str(round(END_TIME - START_TIME, ndigits=2))}s")


Execution Time: 0.13s
